# **DataFrame imdb**

## **Import des bibliothèques nécessaires**

In [1]:
import pandas as pd

## **Chargement des datasets IMDb**

In [4]:
# Nous importons les données directement depuis les fichiers IMDb compressés au format `.tsv.gz`.

#* Datasets principaux
df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_ratings = pd.read_csv('https://datasets.imdbws.com/title.ratings.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

#* Datasets complémentaires
df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_crew = pd.read_csv('https://datasets.imdbws.com/title.crew.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_name_basics = pd.read_csv('https://datasets.imdbws.com/name.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
df_title_principals = pd.read_csv('https://datasets.imdbws.com/title.principals.tsv.gz', sep='\t', compression='gzip', na_values='\\N')

C:\Users\koke7\AppData\Local\Temp\ipykernel_5144\3535879025.py:4: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_basics = pd.read_csv('https://datasets.imdbws.com/title.basics.tsv.gz', sep='\t', compression='gzip', na_values='\\N')
C:\Users\koke7\AppData\Local\Temp\ipykernel_5144\3535879025.py:8: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_title_akas = pd.read_csv('https://datasets.imdbws.com/title.akas.tsv.gz', sep='\t', compression='gzip', na_values='\\N')


## **Nettoyage des datasets IMDb**

In [5]:

#! === SETUP ===
#? title_basics

# 1. Garder uniquement les films (`titleType = 'movie'`).
# 2. Supprimer les lignes avec des valeurs manquantes essentielles.
# 3. Filtrer les années de production pour garder uniquement celles entre 1970 et 2025.
# 4. Réduire le dataset aux colonnes essentielles.

df_title_basics_clean = (
    df_title_basics[df_title_basics['titleType'] == 'movie']
    .dropna(subset=['startYear', 'genres'])
    .assign(startYear=lambda x: x['startYear'].astype(int))
    .query('1970 <= startYear <= 2025')
    [['tconst', 'primaryTitle', 'startYear']]
)


#! === SETUP ===
#? title_rating

# 1. Garder uniquement les films ayant plus de 1000 votes.

df_title_ratings_clean = df_title_ratings[df_title_ratings['numVotes'] > 1000]


#! === SETUP ===
#? title_akas

# 1. Garder uniquement les titres traduits en français (`region = 'FR'`).
# 2. Réduire aux colonnes essentielles et renommer pour correspondre au format attendu.

df_title_akas_clean = (
    df_title_akas[df_title_akas['region'] == 'FR']
    [['titleId', 'title']]
    .rename(columns={'titleId': 'tconst', 'title': 'Titre Français'})
)


#! === SETUP ===
#? title_crew

# 1. Remplir les valeurs manquantes dans `directors` avec "Unknown".
# 2. Transformer `directors` en une liste.
# 3. Exploser les réalisateurs en lignes individuelles pour fusionner avec `name.basics`.
# 4. Regrouper les réalisateurs par film sous forme de liste.

df_title_crew_clean = (
    df_title_crew.fillna({'directors': 'Unknown'})
    .assign(directors=lambda x: x['directors'].str.split(','))
    .explode('directors')
    .merge(df_name_basics[['nconst', 'primaryName']], left_on='directors', right_on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)


#! === SETUP ===
#? title_principals

# 1. Filtrer les acteurs uniquement.
# 2. Fusionner avec `name.basics` pour récupérer les noms des acteurs.
# 3. Regrouper les acteurs par film sous forme de liste.

df_title_principals_clean = (
    df_title_principals.dropna(subset=['category'])
    .query("category == 'actor' or category == 'actress'")
    .merge(df_name_basics[['nconst', 'primaryName']], on='nconst', how='left')
    .groupby('tconst')['primaryName'].apply(list).reset_index()
)

## **Fusion des datasets IMDb**

In [6]:

#! === MERGEUP ===
#* Nous fusionnons les datasets nettoyés pour créer un seul dataframe regroupant toutes les informations utiles.

#* Fusion `title.basics` et `title.ratings`
df_merged_v1 = df_title_basics_clean.merge(df_title_ratings_clean, on='tconst')

#* Fusion avec `title.akas`
df_merged_v2 = df_merged_v1.merge(df_title_akas_clean, on='tconst', how='left')

#* Fusion avec `title.crew`
df_merged_v3 = df_merged_v2.merge(df_title_crew_clean, on='tconst', how='left')

#* Ajout des acteurs depuis `title.principals`
df_merged_v3['Acteurs'] = df_merged_v3['tconst'].map(dict(zip(df_title_principals_clean['tconst'], df_title_principals_clean['primaryName'])))

#* Export du dataframe IMDb final
df_merged_v3.to_csv("../data/raw/df_movie.csv", index=False)

# **Initialisation de TMDB**

## **Chargement des données**

In [1]:
import pandas as pd
import requests

# Clé API TMDb
api_key = 'f26ef44bcadc5d6ffa22263ea37741ce'

# === IMPORT ===
df_movie = pd.read_csv('../data/raw/df_movie.csv')
df_tmdb = pd.read_csv('../data/raw/tmdb_full.csv')

In [2]:
df_tmdb

,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1148172,738771,Ghaltet Habibi,0.000,0,Released,1958-04-14,0,0,False,NaN,...,غلطة حبيبي,Salah and his cousin Souad promise to marry ea...,0.600,/8UrFyUw1ARkWpjocuJYB0wX3aRm.jpg,NaN,"Drama, Romance",NaN,NaN,Arabic,NaN
1148173,738772,Bazaar of All Nations,0.000,0,Released,2010-11-26,0,104,False,NaN,...,Bazaar of All Nations,This documentary film about the Bazaar of All ...,0.600,NaN,"Before malls, life was ""Bazaar""",Documentary,White Lyte Productions,NaN,NaN,NaN
1148174,738773,A Night In The Stu,0.000,0,Released,NaN,0,0,False,NaN,...,A Night In The Stu,A Night In The Stu follows young rappers and a...,0.600,NaN,CMON CUZ,"Documentary, Music",NaN,NaN,NaN,NaN
1148175,738774,Touche pas à ma culture?,0.000,0,Released,2021-01-06,0,0,False,/oa1Y1wqeODVQM7zAHNHPlqIVCaI.jpg,...,Touche pas à ma culture?,NaN,1.116,/dYYk2N9PTU8rHhRxU4pKr1XZH8g.jpg,NaN,Documentary,Attraction Images,Canada,French,cultural appropriation


## **Initialisation des datasets**

In [ ]:

#* Nous conservons uniquement les colonnes pertinentes pour le projet.
df_tmdb = df_tmdb[
    ['id', 'backdrop_path', 'budget', 'genres', 'imdb_id', 'original_language', 'overview', 'popularity', 
     'poster_path', 'production_countries', 'release_date', 'revenue', 'runtime', 
     'spoken_languages', 'vote_average', 'vote_count', 'production_companies']
]

#* Renommage des colonnes pour le dataset IMDb (df_movie)
df_movie.rename(columns={
    'tconst': 'ID imdb',
    'primaryTitle': 'Titre Original',
    'averageRating': 'Note imdb',
    'numVotes': 'Votes imdb',
    'title': 'Titre Français',
    'actors': 'Acteurs',
    'primaryName': 'Réalisateur(s)'
}, inplace=True)

#* Mise en forme des colonnes "Réalisateur(s)" et "Acteurs" pour convertir les listes en chaînes lisibles.
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Réalisateur(s)'] = df_movie['Réalisateur(s)'].str.strip("[]").str.replace("'", "")

df_movie['Acteurs'] = df_movie['Acteurs'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Acteurs'] = df_movie['Acteurs'].str.strip("[]").str.replace("'", "")

#* Renommage des colonnes pour le dataset TMDB (df_tmdb)
df_tmdb.rename(columns={
    'id': 'ID tmdb',
    'backdrop_path': 'Image de Fond',
    'budget': 'Budget',
    'genres': 'Genres',
    'overview': 'Synopsis',
    'original_language': 'Langue Originale',
    'popularity': 'Popularité',
    'poster_path': 'Affiche',
    'production_countries': 'Pays de Production',
    'release_date': 'Date de Sortie',
    'revenue': 'Box Office',
    'runtime': 'Durée',
    'spoken_languages': 'Langues Parlées',
    'vote_average': 'Note tmdb',
    'vote_count': 'Votes tmdb',
    'production_companies': 'Compagnies de Production'
}, inplace=True)

#* Mise en forme des colonnes avec des listes pour les rendre lisibles.
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Compagnies de Production'] = df_tmdb['Compagnies de Production'].str.strip("[]").str.replace("'", "")

df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Langues Parlées'] = df_tmdb['Langues Parlées'].str.strip("[]").str.replace("'", "")

df_tmdb['Genres'] = df_tmdb['Genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_tmdb['Genres'] = df_tmdb['Genres'].str.strip("[]").str.replace("'", "")

df_movie['Acteurs'] = df_movie['Acteurs'].fillna('').str.split(',').apply(set)
df_movie['Acteurs'] = df_movie['Acteurs'].apply(lambda x: ','.join(set(x)))

df_movie['Mots-Clés'] = df_movie['Mots-Clés'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_movie['Mots-Clés'] = df_movie['Mots-Clés'].str.replace(r"['\[\]]", '', regex=True)


C:\Users\koke7\AppData\Local\Temp\ipykernel_5144\4040328731.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmdb.rename(columns={
C:\Users\koke7\AppData\Local\Temp\ipykernel_5144\4040328731.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmdb['Pays de Production'] = df_tmdb['Pays de Production'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
C:\Users\koke7\AppData\Local\Temp\ipykernel_5144\4040328731.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexe

## **Fusion de Imdb et Tmdb**

In [86]:

#! === MERGEUP ===

df_final = pd.merge(df_movie, df_tmdb, left_on='ID imdb', right_on='imdb_id')

## **Ajout de données supplémentaires**

In [98]:

#* Cette colonne catégorise les films en fonction de leur nombre de votes IMDb.
def categorize_votes(votes):
    if votes < 50000:
        return "Connu"
    elif 50000 <= votes < 200000:
        return "Populaire"
    elif 200000 <= votes < 1000000:
        return "Très populaire"
    else:
        return "Blockbuster"

df_final['Réputation'] = df_final['Votes imdb'].apply(categorize_votes)


#* Cette colonne catégorise les films selon leur durée en minutes.
def categorize_times(duree):
    if duree < 100:
        return "Court"
    elif 100 <= duree <= 200:
        return "Moyen"
    else:
        return "Long"

df_final['Métrage'] = df_final['Durée'].apply(categorize_times)


#* Cette colonne regroupe les films par décennie de sortie.
def categorize_years(year):
    if 1970 <= year <= 1979:
        return "70°s"
    if 1980 <= year <= 1989:
        return "80°s"
    if 1990 <= year <= 1999:
        return "90°s"
    if 2000 <= year <= 2009:
        return "2000"
    if 2010 <= year <= 2019:
        return "2010"
    if 2020 <= year <= 2029:
        return "2020"


#* Fonction pour récupérer les mots-clés d'un film par ID
def get_movie_keywords(movie_id):
    url_keywords = f'https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={api_key}&language=en-US'
    response_keywords = requests.get(url_keywords)
    keywords_data = response_keywords.json()
    keywords = [kw['name'] for kw in keywords_data.get('keywords', [])]
    return keywords


#* Ajouter les mots-clés à chaque film dans le DataFrame TMDb complet
df_final['Mots-Clés'] = df_final['ID tmdb'].apply(get_movie_keywords)


df_final['Décennie'] = df_final['Date de Sortie'].apply(
    lambda x: categorize_years(int(str(x)[:4])) if isinstance(x, str) else x
)


#* Cette colonne extrait le premier genre de la liste des genres.
df_final['Genre Principal'] = df_final['Genres'].apply(
    lambda x: x.split(',')[0] if isinstance(x, str) else x
)


#* On exclut les films dont le genre principal est 'Documentary'.
df_final = df_final[df_final['Genre Principal'] != 'Documentary']


#* On repositionne les colonnes pour une meilleure lisibilité et cohérence.
df_final = df_final[
    ['ID imdb', 'ID tmdb', 'Titre Original', 'Titre Français', 'Réalisateur(s)', 'Acteurs',
     'Budget', 'Genres', 'Mots-Clés', 'Genre Principal', 'Date de Sortie', 'Décennie',
     'Langue Originale', 'Langues Parlées', 'Synopsis', 'Popularité', 'Réputation',
     'Affiche', 'Image de Fond', 'Durée', 'Métrage',
     'Note tmdb', 'Votes tmdb', 'Note imdb', 'Votes imdb',
     'Compagnies de Production', 'Pays de Production', 'Box Office']
]


## **Filtrage Général**

In [113]:

# S'assurer que la colonne 'Langues Parlées' est correctement formatée
df_final['Langues Parlées'] = df_final['Langues Parlées'].astype(str)

# Critères de sélection avec contains
df_movie_cleaned_v1 = df_final[
    # Films en français avec plus de 2000 votes et popularité > 1.5
    ((df_final['Votes imdb'] > 8000) & (df_final['Popularité'] > 5) & (df_final['Langue Originale'].str.contains('fr', case=False, na=False))) |
    # Films avec des langues parlées contenant 'French', plus de 10000 votes et popularité > 5
    ((df_final['Votes imdb'] > 15000) & (df_final['Popularité'] > 7.5) & (df_final['Langues Parlées'].str.contains('French', case=False, na=False))) |
    # Films en anglais avec plus de 20000 votes et popularité > 10
    ((df_final['Votes imdb'] > 20000) & (df_final['Popularité'] > 10) & (df_final['Langue Originale'].str.contains('en', case=False, na=False))) |
    # Films populaires dans toutes les langues avec au moins 30000 votes et popularité > 20
    ((df_final['Votes imdb'] > 50000) & (df_final['Popularité'] > 25))
]

#* On conserve uniquement les films ayant des notes IMDb et TMDb supérieures à 4.
df_movie_cleaned_v2 = df_movie_cleaned_v1[
    ~((df_movie_cleaned_v1['Note imdb'] < 4) & (df_movie_cleaned_v1['Note tmdb'] < 4) & (df_movie_cleaned_v1['Date de Sortie'] < '2024-01-01'))
]

#* Exclusion des films ayant une note IMDb inférieure à 8 et sortis avant l'année 2000.
df_movie_cleaned_v3 = df_movie_cleaned_v2[
    ~((df_movie_cleaned_v2['Note imdb'] < 8) & (df_movie_cleaned_v2['Date de Sortie'] < '2000-01-01'))
]

#* On élimine les doublons en se basant sur le titre original.
df_movie_cleaned_v4 = df_movie_cleaned_v3.drop_duplicates(subset='Titre Original')

#* Création d'une copie explicite pour éviter des avertissements potentiels de pandas.
df_movie_cleaned_v4 = df_movie_cleaned_v4.copy()

#* Conversion de la colonne 'Durée' en format numérique.
df_movie_cleaned_v4['Durée'] = pd.to_numeric(df_movie_cleaned_v4['Durée'], errors='coerce')

## **Exportation**

In [115]:

#! === EXPORT ===

df_movie_cleaned_v4.to_csv("../data/processed/df_movie_cleaned.csv", index=False)

## **Rendu :**

In [3]:
import pandas as pd

df = pd.read_csv("../data/processed/df_movie_cleaned.csv")

In [4]:
df

,ID imdb,ID tmdb,Titre Original,Titre Français,Réalisateur(s),Acteurs,Budget,Genres,Mots-Clés,Genre Principal,...,Image de Fond,Durée,Métrage,Note tmdb,Votes tmdb,Note imdb,Votes imdb,Compagnies de Production,Pays de Production,Box Office
0,tt0035423,11232,Kate & Leopold,Kate et Léopold,James Mangold,"Breckin Meyer, Hugh Jackman, Paxton Whitehead...",48000000,"Romance, Fantasy, Comedy","new york city, time travel, duke, fish out of ...",Romance,...,/hfeiSfWYujh6MKhtGTXyK3DD4nN.jpg,118,Moyen,6.328,1232,6.4,90792,"Konrad Pictures, Miramax",United States of America,76019048
1,tt0066921,185,A Clockwork Orange,Orange mécanique,Stanley Kubrick,"Warren Clarke, Paul Farrell, Michael Gover, M...",2200000,"Science Fiction, Drama","london, england, robbery, street gang, great b...",Science Fiction,...,/yIonSXf1jdNihhK8PssxO6KCIfz.jpg,137,Moyen,8.200,11946,8.2,899189,"Warner Bros. Pictures, Hawk Films","United Kingdom, United States of America",26589000
2,tt0067093,14811,Fiddler on the Roof,Un violon sur le toit,Norman Jewison,"Topol, Rosalind Harris, Leonard Frey, Norma Cr...",9000000,"Drama, Romance","dream, parent child relationship, tradition, m...",Drama,...,/rqnOgJWvlURkcOWfGqiACvkTAqG.jpg,179,Moyen,7.686,501,8.0,48886,"United Artists, Cartier Productions, The Miris...",United States of America,83304330
3,tt0067328,25188,The Last Picture Show,La dernière séance,Peter Bogdanovich,"Ellen Burstyn, Eileen Brennan, Sam Bottoms, C...",1300000,Drama,"small town, new love, based on novel or book, ...",Drama,...,/fvmbYtOzQJqj0Y60UwyIB9iAaMD.jpg,119,Moyen,7.629,596,8.0,53932,"BBS Productions, Columbia Pictures, Last Pictu...",United States of America,29133000
4,tt0068646,238,The Godfather,Le Parrain,Francis Ford Coppola,"Robert Duvall, Al Lettieri, Diane Keaton, Ric...",6000000,"Drama, Crime","based on novel or book, loss of loved one, lov...",Drama,...,/tmU7GeKVybMWFButWEGl2M4GeiP.jpg,175,Moyen,8.707,18677,9.2,2076685,"Paramount, Alfran Productions",United States of America,245066411
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4464,tt9848626,585083,Hotel Transylvania 4: Transformania,Hôtel Transylvanie : Changements monstres,"Derek Drymon, Jennifer Kluska","Molly Shannon, Kathryn Hahn, Selena Gomez, Fr...",0,"Animation, Comedy, Family, Adventure, Fantasy","monster, vampire, transformation, aftercredits...",Animation,...,/ep1urICLqtwwV1sTPHP9WXV4Uto.jpg,92,Court,6.998,1642,6.0,41933,"Sony Pictures Animation, Sony Pictures, MRC, C...",United States of America,0
4465,tt9860728,623195,Falling Inn Love,Falling Inn Love,Roger Kumble,"Daniel Watterson, Jeffrey Bowyer-Chapman,Chri...",0,"Romance, Comedy","san francisco, california, new zealand, romanc...",Romance,...,/lVJdFZc8hBJRVnOIay60tl04Qjv.jpg,98,Court,6.521,1090,5.7,22944,MarVista Entertainment,United States of America,0
4466,tt9866072,615665,Holidate,Holidate,John Whitesell,"Cynthy Wu, Kristin Chenoweth, Luke Bracey, Ma...",0,"Comedy, Romance","date, holiday, single, family dinner, family t...",Comedy,...,/8veOfB9RbSzFki0Rq3IQIGsFfhC.jpg,104,Moyen,7.053,1970,6.2,82055,Wonderland Sound and Vision,United States of America,0
4467,tt9873892,736769,They Cloned Tyrone,Ils ont cloné Tyrone,Juel Taylor,"John Boyega, Teyonah Parris, Kiefer Sutherland...",47000000,"Comedy, Science Fiction, Mystery","grave, conspiracy, cynical, disturbed, angry, ...",Comedy,...,/av2wp3R978lp1ZyCOHDHOh4FINM.jpg,122,Moyen,6.775,355,6.6,43648,"MACRO, Made With Love Media",United States of America,0
